In [1]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys
import import_ipynb
import actual


logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

importing Jupyter notebook from actual.ipynb
2019-06-04 12:49:40,193 DEBUG : Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
%%capture
import os
import site


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np


In [3]:
n_examples = 20
n_time_steps = 100
n_width = 11
n_height = 19
inputs = th.ones(n_examples, 1, n_time_steps, n_width * n_height)

In [4]:
device='cuda'

In [5]:
from __future__ import print_function
import torch
import torchvision

In [52]:
dir(torch.dtype)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'is_floating_point']

In [56]:
traindata = (torch.tensor(actual.images, dtype=torch.float).view(811, 11, 1, 19, 225), torch.tensor(actual.labels))

# trainloader = torch.utils.data.DataLoader(trainset, batch_size=12,
#                                           shuffle=True, num_workers=4)


testdata = (torch.tensor(actual.images_test, dtype=torch.float).view(160, 11, 1, 19, 225), torch.tensor(actual.labels_test))

# testloader = torch.utils.data.DataLoader(testset, batch_size=12,
#                                         shuffle=False, num_workers=4)


In [63]:
n_filters = 15
n_filters_2 = 32
n_filters_3 = 64
n_filters_4 = 128

# continue: process time dimension several steps,
# then remove it by meaning across time
# -> n_examples x n_conv_out_filters x (width * height)
# -> transform back to:
# n_examples x n_conv_out_filters x width x height
# -> now it is a normal "image" problem

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, n_filters, (1, 21))
        self.conv2 = nn.Conv2d(n_filters, n_filters_2, (1, 21))
        self.conv3 = nn.Conv2d(n_filters_2, n_filters_3, (1, 21))
        # self.conv4 = nn.Conv2d(n_filters_3, n_filters_4, (1, 21))
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(n_filters_4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc4 = nn.Linear(10, 4)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        # x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        # x = x.view(-1, self.num_flat_features(x))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 15, kernel_size=(1, 21), stride=(1, 1))
  (conv2): Conv2d(15, 32, kernel_size=(1, 21), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(1, 21), stride=(1, 1))
  (fc1): Linear(in_features=128, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=4, bias=True)
)

In [64]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [65]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for e in traindata:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = e[0].to(device), e[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: size mismatch, m1: [11 x 1280], m2: [128 x 120] at /opt/conda/conda-bld/pytorch_1556653114079/work/aten/src/THC/generic/THCTensorMathBlas.cu:268